### GOAL

Build one tiny backend resource using correct thinking order.

In [1]:
"""
    1. Resources -> User
"""

'\n    1. Resources -> User\n'

In [ ]:
"""
    2. Database Model
"""

import uuid
from sqlmodel import Field, SQLModel

class User(SQLModel, table=True):
    __tablename__="users"
    uid: uuid.UUID | None = Field(default=None, primary_key=True)
    email : str = Field(..., unique=True, nullable=False, index=True)
    full_name: str = Field(...)
    hash_psswd: str = Field(..., nullable=False)

In [19]:
"""
    3. Schema
"""
import uuid
from pydantic import BaseModel

class UserCreate(BaseModel):
    email: str
    full_name: str
    hash_psswd: str
    
class UserRead(BaseModel):
    uid: uuid.UUID
    email: str
    full_name: str

In [20]:
"""
    4. Service
"""

from sqlmodel import select
from sqlalchemy.ext.asyncio import AsyncSession


class UserService:
    async def create(self, payload: UserCreate, session: AsyncSession):
        statement = select(User).where(User.email == payload.email)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if user:
            return {"error":"User already exists"}
        
        new_user = User(email = payload.email,
                        full_name = payload.full_name,
                        password = payload.hash_psswd)
        
        
        session.add(new_user)
        session.commit()
        session.refresh(new_user)
        
        return new_user

In [21]:
def get_session(ab):
    pass

In [22]:
"""
    4. Routes (REST Endpoints)
"""


from fastapi import FastAPI, status, Depends, HTTPException
from sqlalchemy.ext.asyncio import AsyncSession

app = FastAPI()
user_service = UserService()

@app.post("/create", response_model=UserRead, status_code=status.HTTP_201_CREATED)
async def create_user(payload: UserCreate, session: AsyncSession = Depends(get_session)):
    try:
        user = user_service.create(payload, session)
        return user
    
    except ValueError as e:
        raise HTTPException(status_code=status.HTTP_400_BAD_REQUEST, detail=str(e))